# 2. Model Dojo

In short, this notebook dedicated for SRCNN models training phase.

In [31]:
# Importing Packages
import os
import tensorflow
import numpy
from glob import glob
from tensorflow.keras import Model
from tensorflow.keras.layers import ReLU, Conv2D, Input
from tensorflow.keras.optimizers import Adam

### 2.1 Taking Data to be ready 

Currently I'm only have a low amount of training data set. Hence I will only make sure the code is working properly right now and saving the actual training process later. So, I will kind of using all the datasets to be trained of.

In [32]:
def data_preparation():
    input = [*glob("../resources/np_image_input/*")]
    output = [*glob("../resources/np_image_output/*")]

    print(len(input), len(output))
    input.sort()
    output.sort()

    return input, output

In [44]:
class PatchesDataset(tensorflow.keras.utils.Sequence):
    def __init__(self, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.input = [*glob('../resources/np_image_input/*')]
        self.output = [*glob('../resources/np_image_output/*')]
        self.input.sort()
        self.output.sort()
        self.total_data = len(self.input)

    def __len__(self):
        # returns the number of batches
        return int(self.total_data / self.batch_size)

    def __getitem__(self, index):
        # returns one batch
        indices = self.random_indices()
        input = numpy.array([numpy.load(self.input[idx]) for idx in indices])
        output = numpy.array([numpy.load(self.output[idx]) for idx in indices])
        return input, output

    def random_indices(self):
        return numpy.random.choice(list(range(self.total_data)), self.batch_size, p=numpy.ones(self.total_data)/self.total_data)

### 2.2 Model Definition

I can't explain why is this like this... Unfortunately...

in short, this step will create a model, create an optimization through it, and create model summary

Dzlab said *"The architecture of the SRCNN model is very simple, it has only convolutional layers, one to downsize the input and extract image features and a later one to upside to generate the output image. The following helper function is used to create an instance of the model."*

In [34]:
def create_model(height, width, depth):
    input = Input(shape=(height, width, depth))
    x = Conv2D(filters=64, kernel_size=(9, 9), kernel_initializer='he_normal')(input)
    x = ReLU()(x)
    x = Conv2D(filters=32, kernel_size=(1, 1), kernel_initializer='he_normal')(x)
    x = ReLU()(x)
    output = Conv2D(filters=depth, kernel_size=(5, 5), kernel_initializer='he_normal')(x)
    return Model(input, output)

In [35]:
EPOCHS = 12
INPUT_DIM = 33 # This came from the 1.data_processing.ipynb
optimizer = Adam(learning_rate = 1e-3, decay = 1e-3 / EPOCHS)
model = create_model(INPUT_DIM, INPUT_DIM, 3)
model.compile(loss = 'mse', optimizer = optimizer)

In [36]:
# See model summary
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 33, 33, 3)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 25, 25, 64)        15616     
                                                                 
 re_lu_6 (ReLU)              (None, 25, 25, 64)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 25, 25, 32)        2080      
                                                                 
 re_lu_7 (ReLU)              (None, 25, 25, 32)        0         
                                                                 
 conv2d_11 (Conv2D)          (None, 21, 21, 3)         2403      
                                                                 
Total params: 20,099
Trainable params: 20,099
Non-trainable

In [37]:
# Plot the model
tensorflow.keras.utils.plot_model(model, show_shapes = True, rankdir='LR')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [38]:
# Save model weight
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

### 2.3 Fitting the Model
Let's fit the model

In [45]:
# model.fit(data_preparation(), epochs=EPOCHS, callbacks=[cp_callback])

train_dataset = PatchesDataset(1024)
print(len(train_dataset))
# model.fit(train_dataset, epochs=EPOCHS, callbacks=[cp_callback])

0


## 2.4 Save the model to be deployed on the server

In [ ]:
# path = '../resources/suhu/shifu.mdl'
# model.save(path)
# new_model = tf.keras.models.load_model(path)